In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
train = pd.read_csv('raw_data/Train_data.csv')
test = pd.read_csv('raw_data/Test_data.csv')
train_header = train.columns
test_header = test.columns
train = np.array(train)
test = np.array(test)
print(train_header)
print(test_header)

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')
Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')


In [3]:
# processing language feature
lang = train[:, 6]
unique_element = np.unique(lang)
print('unique_element:\n', unique_element, '\n')

lang_count = Counter(lang)
lang_count = dict(lang_count)
lang_count = sorted(lang_count.items())
lang_count = dict(lang_count)
print('lang_count:\n', lang_count, '\n')

lang_avg_rating = []
for i in range(len(unique_element)):
    lang_avg_rating.append(np.mean([train[j, 3] for j in range(len(lang)) if lang[j] == unique_element[i]]))
lang_avg_rating = np.around(lang_avg_rating, 2)
print('lang_avg_rating:\n', lang_avg_rating, '\n')
total_avg_rating = np.mean(lang_avg_rating)
total_avg_rating = np.around(total_avg_rating, 2)
print('total_avg_rating:\n', total_avg_rating, '\n')

# use Bayesian average
bayesian_avg_lang_rating = []
for i in range(len(unique_element)):
    element_num = lang_count[unique_element[i]]
    bayesian_avg_lang_rating.append(5/(5+element_num)*total_avg_rating + (element_num/(element_num+5))*lang_avg_rating[i])
bayesian_avg_lang_rating = np.around(bayesian_avg_lang_rating, 2)
print('bayesian_avg_lang_rating:\n', bayesian_avg_lang_rating, '\n')

# replace all language feature of trainset and testset
for i in range(len(unique_element)):
    for j in range(len(lang)):
        if lang[j] == unique_element[i]:
            lang[j] = bayesian_avg_lang_rating[i]
train[:, 6] = lang

# replace test set language
lang_test = test[:, 6]
unique_element_test = np.unique(lang_test)
print('unique_element_test:\n', unique_element_test, '\n')

for i in range(len(unique_element_test)):
    for j in range(len(lang_test)):
        if lang_test[j] == unique_element[i]:
            lang_test[j] = bayesian_avg_lang_rating[i]
for j in range(len(lang_test)):
    if isinstance(lang_test[j], str):
        lang_test[j] = total_avg_rating

test[:, 6] = lang_test

unique_element:
 ['ara' 'en-CA' 'en-GB' 'en-US' 'eng' 'enm' 'fre' 'ger' 'gla' 'glg' 'grc'
 'ita' 'jpn' 'lat' 'msa' 'mul' 'nl' 'nor' 'por' 'rus' 'spa' 'srp' 'swe'
 'tur' 'wel' 'zho'] 

lang_count:
 {'ara': 1, 'en-CA': 6, 'en-GB': 189, 'en-US': 1285, 'eng': 8001, 'enm': 3, 'fre': 130, 'ger': 85, 'gla': 1, 'glg': 1, 'grc': 10, 'ita': 4, 'jpn': 44, 'lat': 3, 'msa': 1, 'mul': 19, 'nl': 1, 'nor': 1, 'por': 9, 'rus': 2, 'spa': 187, 'srp': 1, 'swe': 2, 'tur': 1, 'wel': 1, 'zho': 12} 

lang_avg_rating:
 [3.55 4.03 3.93 3.92 3.93 3.87 3.98 3.95 4.47 3.36 3.62 4.1  4.27 4.35
 4.11 4.13 4.18 3.6  3.96 4.26 3.95 0.   3.46 4.42 5.   4.53] 

total_avg_rating:
 3.88 

bayesian_avg_lang_rating:
 [3.82 3.96 3.93 3.92 3.93 3.88 3.98 3.95 3.98 3.79 3.71 3.98 4.23 4.06
 3.92 4.08 3.93 3.83 3.93 3.99 3.95 3.23 3.76 3.97 4.07 4.34] 

unique_element_test:
 ['ale' 'en-CA' 'en-GB' 'en-US' 'eng' 'fre' 'ger' 'grc' 'ita' 'jpn' 'por'
 'spa' 'zho'] 



In [4]:
# preprocessing author feature
author_train = train[:, 2]
author_all = []
author_unique = []
for i in range(len(author_train)):
    authors = author_train[i].split('/')
    for j in range(len(authors)):
        author_all.append(authors[j])
author_unique = list(set(author_all))
author_unique = sorted(author_unique)
print('author total num:', len(author_unique), '\n')

# count author num
author_num = Counter(author_all)
author_num = dict(author_num)
author_num = sorted(author_num.items())
author_num = dict(author_num)

# count author rating
author_rating = []
for i in range(len(author_unique)):
    author_rating.append(np.mean([train[j, 3] for j in range(len(author_train)) if author_unique[i] in author_train[j]]))

# count total average rating of author
total_avg_author_rating = np.mean(author_rating)
total_avg_author_rating = np.around(total_avg_author_rating, 2)
print('total_avg_author_rating:', total_avg_author_rating, '\n')

# count author bayesian average rating
bayesian_avg_author_rating = []
for i in range(len(author_unique)):
    element_num = author_num[author_unique[i]]
    bayesian_avg_author_rating.append(element_num/(element_num+2)*author_rating[i] + 2/(element_num+2)*total_avg_author_rating)
bayesian_avg_author_rating = np.around(bayesian_avg_author_rating, 2)
print('bayesian_avg_author_rating:', bayesian_avg_author_rating, '\n')

# replace train set author feature
for i in range(len(author_train)):
    authors = author_train[i].split('/')
    score = 0
    for j in range(len(authors)):
        score += bayesian_avg_author_rating[author_unique.index(authors[j])]
    author_train[i] = score/len(authors)
author_train = np.around(np.float64(author_train), 2)
train[:, 2] = author_train

# replace test set author feature
author_test = test[:, 2]
for i in range(len(author_test)):
    authors = author_test[i].split('/')
    score = []
    for j in range(len(authors)):
        if authors[j] in author_unique:
            score.append(bayesian_avg_author_rating[author_unique.index(authors[j])])
        else:
            score.append(total_avg_author_rating)
    author_test[i] = np.mean(score)
author_test = np.around(np.float64(author_test), 2)
test[:, 2] = author_test


author total num: 8535 

total_avg_author_rating: 3.92 

bayesian_avg_author_rating: [4.03 3.95 3.92 ... 4.04 4.43 4.14] 



In [7]:
# write to file
train = pd.DataFrame(train)
train.to_csv('./processed_data/train_processed_language_author.csv', header=train_header)

test = pd.DataFrame(test)
test.to_csv('./processed_data/test_processed_language_author.csv', header=test_header)


In [ ]:
# # 舍弃isbn，因为提取出来的info与language重合了
# isbn = train[:, 4]
# delete_index = []
# for i in range(len(isbn)):
#     isbn[i] = isbnlib.to_isbn10(str(isbn[i]))
#     if isbn[i] == '':
#         delete_index.append(i)
# isbn = [isbn[i] for i in range(len(isbn)) if i not in delete_index]
# print(len(isbn))
# isbn_info = []
# for i in range(len(isbn)):
#     isbn_info.append(isbnlib.info(isbn[i]))
#     print(isbn_info[i])